# 연관규칙분석 (association rule)
Market Basket Analysis 이라고도 하며, Apriori 알고리즘이 가장 많이 사용된다

In [1]:
#실습
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
 
    
dataset = [['Milk', 'Onion', 'Nutmeg', 'Eggs', 'Yogurt'],
           ['Onion', 'Nutmeg', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Ice cream', 'Eggs']]

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)



In [3]:
df

,Apple,Corn,Eggs,Ice cream,Milk,Nutmeg,Onion,Unicorn,Yogurt
0,False,False,True,False,True,True,True,False,True
1,False,False,True,False,False,True,True,False,True
2,True,False,True,False,True,False,False,False,False
3,False,True,False,False,True,False,False,True,True
4,False,True,True,True,False,False,True,False,False


In [2]:
frequent_itemsets = apriori(df, min_support=0.5, use_colnames=True)

In [4]:
frequent_itemsets

,support,itemsets
0,0.8,(Eggs)
1,0.6,(Milk)
2,0.6,(Onion)
3,0.6,(Yogurt)
4,0.6,"(Eggs, Onion)"


5개의 장바구니 중 eggs는 4번 들어가있으므로 0.8의 확률, 즉 itemsets는 물건의 조합, support는 그 물건의 조합이 들어있을 확률

In [5]:
association_rules(frequent_itemsets, metric="lift", min_threshold=1)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Eggs),(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.6
1,(Onion),(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf


support가 0.5가 넘는 항목에 대해 향상도가 양의 상관관계에 있는것. eggs와 onion

------------

## Data load

In [5]:
import pandas as pd
import pandas as pd
import json
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [7]:
datapath = "D:/Jihye/data/"

#csv파일 불러와서 dataframe화
with open(datapath+"genre_gn_all.json", encoding = 'utf-8') as json_file: genre = json.load(json_file)
with open(datapath+"song_meta.json", encoding = 'utf-8') as json_file: song_meta = json.load(json_file)
with open(datapath+"train.json", encoding = 'utf-8') as json_file: train = json.load(json_file)
with open(datapath+"val.json", encoding = 'utf-8') as json_file: valid = json.load(json_file)
    
    
# json to dataframe
df_genre = pd.Series(genre).to_frame('genre').reset_index().rename({'index' : 'gen_id'}, axis = 1)
df_song_meta = pd.DataFrame(song_meta)
df_train = pd.DataFrame(train)
df_valid = pd.DataFrame(valid)


In [6]:
#songmeta에서 장르만 추출
gnr_meta=df_song_meta[['song_gn_dtl_gnr_basket','id']]

#playlist의 songs만 추출
playlist_songs=df_train['songs']

#각 playlist에 들어있는 장르만 추출 -> dataset
c=0
dataset_gnr=[]
for p in playlist_songs:
    gnr_list=[]
    for song in p:
        g=gnr_meta.loc[song,'song_gn_dtl_gnr_basket']
        
        try: #결측값 패스
            if len(g)==1:
                gnr_list.append(g[0])
            else:            
                for i in g:
                    gnr_list.append(i)
        except: pass    
    dataset_gnr.append(gnr_list)
    
    c+=1
    #print(c)
    if c==30:
        break #엄청빠름

In [7]:
dataset_gnr

[['GN1402',
  'GN1401',
  'GN0901',
  'GN0902',
  'GN1001',
  'GN1012',
  'GN1005',
  'GN1001',
  'GN1013',
  'GN0901',
  'GN0902',
  'GN1001',
  'GN1013',
  'GN0901',
  'GN0902',
  'GN1001',
  'GN1013',
  'GN0901',
  'GN0902',
  'GN1001',
  'GN1003',
  'GN1013',
  'GN1001',
  'GN0901',
  'GN1304',
  'GN1301',
  'GN1302',
  'GN0904',
  'GN1402',
  'GN1401',
  'GN0901',
  'GN1001',
  'GN1001',
  'GN1912',
  'GN1904',
  'GN1901',
  'GN0901',
  'GN0901',
  'GN0902',
  'GN1001',
  'GN1402',
  'GN1401',
  'GN0901',
  'GN1003',
  'GN1001',
  'GN0901',
  'GN1102',
  'GN1101'],
 ['GN0101',
  'GN0103',
  'GN0601',
  'GN0605',
  'GN0104',
  'GN0101',
  'GN0601',
  'GN0605',
  'GN0104',
  'GN0101',
  'GN0601',
  'GN0605',
  'GN0104',
  'GN0101',
  'GN0508',
  'GN0501',
  'GN0601',
  'GN0503',
  'GN0605',
  'GN0508',
  'GN0501',
  'GN0601',
  'GN0503',
  'GN0605',
  'GN0805',
  'GN0501',
  'GN0502',
  'GN0801',
  'GN0509',
  'GN0805',
  'GN0501',
  'GN0502',
  'GN0801',
  'GN0509',
  'GN0105',
  '

In [4]:
#print (dataset_gnr)

#association_rules 분석을 위한 one-hot encoding
te = TransactionEncoder()
te_ary = te.fit(dataset_gnr).transform(dataset_gnr)
result = pd.DataFrame(te_ary, columns=te.columns_)

In [5]:
result.head()

,GN0101,GN0103,GN0104,GN0105,GN0201,GN0203,GN0204,GN0205,GN0301,GN0302,...,GN2504,GN2505,GN2506,GN2601,GN2602,GN2603,GN2703,GN2704,GN2901,GN2902
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,True,True,True,False,False,False,False,False,False,...,False,False,False,True,True,True,False,False,False,False
2,True,False,False,False,False,False,False,False,True,False,...,False,False,False,True,False,True,False,False,False,False
3,True,True,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,True,True,False,False,True,True,True,False,True,True,...,True,True,True,False,False,False,False,False,False,False


## Apriori 알고리즘 적용

In [12]:
frequent_itemsets = apriori(result, min_support=0.4, use_colnames=True) #Apriori 적용, 전체에서 등장확률이 min_support이상인 itemset만 출력
frequent_itemsets

,support,itemsets
0,0.600000,(GN0101)
1,0.466667,(GN0105)
2,0.433333,(GN0201)
3,0.500000,(GN0301)
4,0.466667,(GN0303)
5,0.400000,(GN0401)
6,0.466667,(GN0501)
7,0.466667,(GN0509)
8,0.533333,(GN0601)
9,0.433333,(GN0606)


In [13]:
#위의 결과에서 min_support 이상인 itemset에 대해 연관 규칙 결과 보여주기
rules=association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(GN0101),(GN0105),0.600000,0.466667,0.466667,0.777778,1.666667,0.186667,2.4
1,(GN0105),(GN0101),0.466667,0.600000,0.466667,1.000000,1.666667,0.186667,inf
2,(GN0101),(GN0301),0.600000,0.500000,0.400000,0.666667,1.333333,0.100000,1.5
3,(GN0301),(GN0101),0.500000,0.600000,0.400000,0.800000,1.333333,0.100000,2.0
4,(GN0101),(GN0303),0.600000,0.466667,0.400000,0.666667,1.428571,0.120000,1.6
...,...,...,...,...,...,...,...,...,...
101,"(GN0801, GN0509)","(GN0805, GN0501)",0.400000,0.400000,0.400000,1.000000,2.500000,0.240000,inf
102,(GN0805),"(GN0801, GN0501, GN0509)",0.400000,0.400000,0.400000,1.000000,2.500000,0.240000,inf
103,(GN0501),"(GN0805, GN0509, GN0801)",0.466667,0.400000,0.400000,0.857143,2.142857,0.213333,4.2
104,(GN0509),"(GN0805, GN0501, GN0801)",0.466667,0.400000,0.400000,0.857143,2.142857,0.213333,4.2


#### rules 분석

1. 어떤 플레이리스트에 GN0805,GN0101,GN0301이 있다고 할 때

-각 열은 frozenset이라는 변경할 수 없는 집합 자료형으로, list와 dict처럼 특정요소를 출력하는게 불가능한 대신 특정 요소가 set안에 포함되어있는지 in, not in 구문으로 True, False값을 반환

In [101]:
playlist=['GN0805','GN0101','GN0301']
#rules[(rules['antecedents']=={playlist[0]}) | (rules['antecedents']=={playlist[1]}) | (rules['antecedents']=={playlist[2]})]
#rules[rules['antecedents']=={p for p in playlist}]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(GN0101),(GN0105),0.6,0.466667,0.466667,0.777778,1.666667,0.186667,2.400000
2,(GN0101),(GN0301),0.6,0.500000,0.400000,0.666667,1.333333,0.100000,1.500000
3,(GN0301),(GN0101),0.5,0.600000,0.400000,0.800000,1.333333,0.100000,2.000000
4,(GN0101),(GN0303),0.6,0.466667,0.400000,0.666667,1.428571,0.120000,1.600000
6,(GN0101),(GN0601),0.6,0.533333,0.466667,0.777778,1.458333,0.146667,2.100000
8,(GN0101),(GN1501),0.6,0.600000,0.433333,0.722222,1.203704,0.073333,1.440000
10,(GN0101),(GN1504),0.6,0.400000,0.400000,0.666667,1.666667,0.160000,1.800000
12,(GN0101),(GN2501),0.6,0.466667,0.400000,0.666667,1.428571,0.120000,1.600000
18,(GN0301),(GN0303),0.5,0.466667,0.466667,0.933333,2.000000,0.233333,8.000000
20,(GN0301),(GN2501),0.5,0.466667,0.400000,0.800000,1.714286,0.166667,2.666667


In [116]:
A=rules[(rules.antecedents=={playlist[0]})]
for p in range(len(playlist)-1):
    B=rules[(rules.antecedents=={playlist[p+1]})]
    C=pd.concat([A,B],axis=0).reset_index(drop=True)
    A=C

selected=A[['antecedents','consequents','support','confidence','lift']].sort_values(by=['lift'],axis=0,ascending=False) #향상도에 내림차순 정렬
selected[selected['lift']>1.5] #향상도 1.5이상만 보여줌

,antecedents,consequents,support,confidence,lift
2,(GN0805),(GN0801),0.400000,1.000000,2.500000
4,(GN0805),"(GN0801, GN0501)",0.400000,1.000000,2.500000
5,(GN0805),"(GN0801, GN0509)",0.400000,1.000000,2.500000
6,(GN0805),"(GN0801, GN0501, GN0509)",0.400000,1.000000,2.500000
0,(GN0805),(GN0501),0.400000,1.000000,2.142857
1,(GN0805),(GN0509),0.400000,1.000000,2.142857
3,(GN0805),"(GN0501, GN0509)",0.400000,1.000000,2.142857
19,(GN0301),"(GN0101, GN0303)",0.400000,0.800000,2.000000
17,(GN0301),(GN0303),0.466667,0.933333,2.000000
20,(GN0301),"(GN2501, GN0303)",0.400000,0.800000,2.000000


1.GN0805장르가 있으면 GN0801,0501,0509이 포함될 확률이 매우 크다. 

2.GN0301장르가 있으면 GN0101,0303,2501이 포함될 확률이 크다.

3.GN0101장르가 있으면 GN1504,1501,0105가 포함될 확률이 조금 크다.

향상도와 신뢰도를 바탕으로 장바구니에 담기 

set은 중복을 허용하지 않는 대신 순서가 없음..

In [138]:
import itertools

S=selected[selected['lift']>1.5].reset_index(drop=True)
genre_list=[]
for c in range(len(S.consequents)):
    G=list(S.consequents[c])
    genre_list.append(G)
    
genre_list=list(itertools.chain.from_iterable(genre_list))        
#genre_list

Genre=[]
for v in genre_list:
    if v not in Genre:
        Genre.append(v)

Genre


['GN0801',
 'GN0501',
 'GN0509',
 'GN0101',
 'GN0303',
 'GN2501',
 'GN1504',
 'GN1501',
 'GN0105']

### 분석 결론

GN0805,GN0101,GN0301 > 포크/블루스 2010년이후발매, 발라드 전체, 랩 힙합 전체 장르와 가장 연관깊은 장르는

포크/블루스 세부장르 전체, 인디음악 세부장르전체, 인디음악 2010년이후발매, 발라드 전체, 랩/힙합 보컬스타일, 아이돌 세부장르전체, 국내드리마 ost, 국내드라마 ost전체, 10년도 이후 발매된 발라드 순으로 나타남

-----------

## tag 분석 적용

In [8]:
df_train

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000
...,...,...,...,...,...,...
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000
115069,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000


In [9]:
# 플레이리스트 아이디(id)와 매핑된 태그(tags) 추출
plylst_tag_map = df_train[['id', 'tags']]

plylst_tag_map

,id,tags
0,61281,[락]
1,10532,"[추억, 회상]"
2,76951,"[까페, 잔잔한]"
3,147456,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,..."
4,27616,[댄스]
...,...,...
115066,120325,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]"
115067,106976,[일렉]
115068,11343,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]"
115069,131982,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]"


In [10]:
tag_list=df_train['tags'].tolist()

In [11]:
te = TransactionEncoder()
te_ary = te.fit(tag_list).transform(tag_list)
result = pd.DataFrame(te_ary, columns=te.columns_)

In [12]:
result

,,00,007,007시리즈,00s,00년,00년대,00년대노래,00년대발라드,00년도노래,...,힙합플레이리스트,힙합플레이야,힙합플레이야2018,힙합홍수,힙합힙합,힙해,힙힙힙,힛뎀포크,힛뎀폭,힛뎀폭스
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115066,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
115067,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
115068,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
115069,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [18]:
frequent_itemsets = apriori(result, min_support=0.01, use_colnames=True) #Apriori 적용, 전체에서 등장확률이 min_support이상인 itemset만 출력

In [19]:
frequent_itemsets

,support,itemsets
0,0.013279,(EDM)
1,0.020188,(OST)
2,0.028226,(Pop)
3,0.035126,(가을)
4,0.099217,(감성)
...,...,...
109,0.011445,"(기분전환, 신나는, 드라이브)"
110,0.011176,"(여행, 기분전환, 드라이브)"
111,0.013531,"(기분전환, 휴식, 힐링)"
112,0.013991,"(밤, 새벽, 잔잔한)"


In [10]:
rules=association_rules(frequent_itemsets, metric="lift", min_threshold=1)


In [25]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(가을),(감성),0.035126,0.099217,0.009186,0.261504,2.635679,0.005701,1.219754
1,(감성),(가을),0.099217,0.035126,0.009186,0.092581,2.635679,0.005701,1.063317
2,(감성),(겨울),0.099217,0.034457,0.009985,0.100639,2.920725,0.006566,1.073588
3,(겨울),(감성),0.034457,0.099217,0.009985,0.289786,2.920725,0.006566,1.268325
4,(기분전환),(감성),0.143086,0.099217,0.024333,0.170058,1.713997,0.010136,1.085356
...,...,...,...,...,...,...,...,...,...
159,"(휴식, 잔잔한)",(힐링),0.024428,0.085017,0.013122,0.537175,6.318441,0.011046,1.976954
160,"(힐링, 잔잔한)",(휴식),0.018562,0.097462,0.013122,0.706929,7.253411,0.011313,3.079588
161,(휴식),"(힐링, 잔잔한)",0.097462,0.018562,0.013122,0.134641,7.253411,0.011313,1.134139
162,(힐링),"(휴식, 잔잔한)",0.085017,0.024428,0.013122,0.154349,6.318441,0.011046,1.153634


어떤 태그에 대해서 연관도가 높은 태그들을 뽑고싶으면 tag_list를 만드는 단계에서 그 태그를 포함한 playlist만 들어가게 해야할듯

하지만 이렇게하면 validation하나 할 때 마다 연관분석을 계속 해야함...어떡하징

In [12]:
tag_list[0]

['락']

In [17]:


def associated_tags(tag_list,rules,i):
    playlist=tag_list[i]
    A=rules[(rules.antecedents=={playlist[0]})]
    for p in range(len(playlist)-1):
        try:
            B=rules[(rules.antecedents=={playlist[p+1]})]
            C=pd.concat([A,B],axis=0).reset_index(drop=True)
            A=C
        except: break
    
    selected=A[['antecedents','consequents','support','confidence','lift']].sort_values(by=['lift'],axis=0,ascending=False) #향상도에 내림차순 정렬
    #selected[selected['lift']>1.5] #향상도 1.5이상만 보여줌

    S=selected[selected['lift']>1.0].reset_index(drop=True)
    taglist=[]
    
    for c in range(len(S.consequents)):
        T=list(S.consequents[c])
        taglist.append(T)
        
    taglist=list(itertools.chain.from_iterable(taglist))   

    Tag=[]
    for v in taglist:
        if v not in Tag:
            Tag.append(v)
     
    return Tag
    

In [39]:
i=0
tag_list=[['카페']]
Tag=associated_tags(tag_list,rules,i)

In [40]:
Tag

['매장음악', '감성', '드라이브', '휴식', '잔잔한', '사랑', '기분전환', '힐링']

## pipeline

In [1]:
import pandas as pd
import pandas as pd
import json
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import itertools

datapath = "D:/Jihye/data/"

In [2]:

def association_tags(datapath): #데이터 경로, min_support값, 반복할 횟수

    #csv파일 불러와서 dataframe화
    with open(datapath+"genre_gn_all.json", encoding = 'utf-8') as json_file: genre = json.load(json_file)
    with open(datapath+"song_meta.json", encoding = 'utf-8') as json_file: song_meta = json.load(json_file)
    with open(datapath+"train.json", encoding = 'utf-8') as json_file: train = json.load(json_file)
    with open(datapath+"val.json", encoding = 'utf-8') as json_file: valid = json.load(json_file)
    
    # json to dataframe
    df_genre = pd.Series(genre).to_frame('genre').reset_index().rename({'index' : 'gen_id'}, axis = 1)
    df_song_meta = pd.DataFrame(song_meta)
    df_train = pd.DataFrame(train)
    df_valid = pd.DataFrame(valid)
    tag_list=df_train['tags'].tolist()
    
    return tag_list, df_train, df_genre, df_song_meta, df_valid


tag_list, df_train, df_genre, df_song_meta, df_valid = association_tags(datapath)

In [3]:
tag_list

[['락'],
 ['추억', '회상'],
 ['까페', '잔잔한'],
 ['연말',
  '눈오는날',
  '캐럴',
  '분위기',
  '따듯한',
  '크리스마스캐럴',
  '겨울노래',
  '크리스마스',
  '겨울왕국',
  '크리스마스송'],
 ['댄스'],
 ['운동', '드라이브', 'Pop', '트로피컬하우스', '힐링', '기분전환', '2017', '팝', '트렌드', '일렉'],
 ['짝사랑', '취향저격', '슬픔', '고백', '사랑', '이별'],
 ['잔잔한', '추억', '회상'],
 ['일렉트로니카', '포크', '메탈', '락', '댄스', '인디'],
 ['록', 'Metal', '이일우', 'M에센셜', '메탈', 'Rock', '락'],
 ['kpop', '댄스', '걸그룹댄스', '스트레스해소'],
 ['새해', '여행', '프로필음악', '카카오톡', '기분전환', '소원', '프로필', '소망', '다짐', '카톡'],
 ['듣고', '우울', '힐링', '이거', '힘내'],
 ['힙합', '느낌있는', '밤', '새벽', 'RnB', '감각적인', '드라이브', '국내', '그루브한'],
 ['가을', '재즈'],
 ['락'],
 ['감성', '질리지않는', '나만알고싶은', 'Pop'],
 ['봄', '설렘', '사랑'],
 ['비오는날', '누군가생각날때'],
 ['스밍', '목록', '폐막식', '올림픽', '엑소'],
 ['조용히', '혼자', '또는', '새벽감성', '고민', '맥주한잔'],
 ['카페', '재즈', '잔잔한'],
 ['OST'],
 ['댄스'],
 ['감성', '어쿠스틱', '잔잔한', '새벽', '편안한', '인디', '밤'],
 ['잔잔한', '밤', '새벽'],
 ['힐링', '드라이브', '에너지', '인디', '여행'],
 ['듀엣', '취향저격', '달달', '피쳐링', '사랑', '남녀'],
 ['까페', '설렘', '사랑'],
 ['힙합', '신인', '국힙', '인디', 

In [4]:
def onehotEncoding(tag_list):
    te = TransactionEncoder()
    te_ary = te.fit(tag_list).transform(tag_list)
    return pd.DataFrame(te_ary, columns=te.columns_)

In [5]:
result=onehotEncoding(tag_list)

In [6]:
result

,,00,007,007시리즈,00s,00년,00년대,00년대노래,00년대발라드,00년도노래,...,힙합플레이리스트,힙합플레이야,힙합플레이야2018,힙합홍수,힙합힙합,힙해,힙힙힙,힛뎀포크,힛뎀폭,힛뎀폭스
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115066,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
115067,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
115068,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
115069,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [7]:
def analysis_tags(result, min_support) :
    frequent_itemsets = apriori(result, min_support=min_support, use_colnames=True) #Apriori 적용, 전체에서 등장확률이 min_support이상인 itemset만 출력
    rules=association_rules(frequent_itemsets, metric="lift", min_threshold=1)
    return frequent_itemsets, rules

In [ ]:
frequent_itemsets, rules = analysis_tags(result, 0.002)

In [9]:
frequent_itemsets

,support,itemsets
0,0.005371,(CCM)
1,0.013279,(EDM)
2,0.005962,(Jazz)
3,0.020188,(OST)
4,0.028226,(Pop)
...,...,...
312,0.013122,"(휴식, 잔잔한, 힐링)"
313,0.005197,"(기분전환, 여행, 드라이브, 신나는)"
314,0.005770,"(슬픔, 이별, 밤, 새벽)"
315,0.005570,"(휴식, 힐링, 밤, 새벽)"


In [ ]:
    def association_rule(rules,i):
        playlist=tag_list[i]
        A=rules[(rules.antecedents=={playlist[0]})]
        for p in range(len(playlist)-1):
            try:
                B=rules[(rules.antecedents=={playlist[p+1]})]
                C=pd.concat([A,B],axis=0).reset_index(drop=True)
                A=C
            except: break
    
        selected=A[['antecedents','consequents','support','confidence','lift']].sort_values(by=['lift'],axis=0,ascending=False) #향상도에 내림차순 정렬
    #selected[selected['lift']>1.5] #향상도 1.5이상만 보여줌

        S=selected[selected['lift']>1.0].reset_index(drop=True)
        taglist=[]
    
        for c in range(len(S.consequents)):
            T=list(S.consequents[c])
            taglist.append(T)
        
        taglist=list(itertools.chain.from_iterable(taglist))   

        Tag=[]
        for v in taglist:
            if v not in Tag:
                Tag.append(v)
     
        return Tag
    
    for i in range(n):
        Tag=association_rule(rules,i)
        print (Tag)
        

In [2]:
import pandas as pd
import pandas as pd
import json
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import itertools

datapath = "D:/Jihye/data/"
def association_tags(datapath,min_support,n): #데이터 경로, min_support값, 반복할 횟수

    #csv파일 불러와서 dataframe화
    with open(datapath+"genre_gn_all.json", encoding = 'utf-8') as json_file: genre = json.load(json_file)
    with open(datapath+"song_meta.json", encoding = 'utf-8') as json_file: song_meta = json.load(json_file)
    with open(datapath+"train.json", encoding = 'utf-8') as json_file: train = json.load(json_file)
    with open(datapath+"val.json", encoding = 'utf-8') as json_file: valid = json.load(json_file)
    
    # json to dataframe
    df_genre = pd.Series(genre).to_frame('genre').reset_index().rename({'index' : 'gen_id'}, axis = 1)
    df_song_meta = pd.DataFrame(song_meta)
    df_train = pd.DataFrame(train)
    df_valid = pd.DataFrame(valid)
    tag_list=df_train['tags'].tolist()
    
    te = TransactionEncoder()
    te_ary = te.fit(tag_list).transform(tag_list)
    result = pd.DataFrame(te_ary, columns=te.columns_)
        
    frequent_itemsets = apriori(result, min_support=min_support, use_colnames=True) #Apriori 적용, 전체에서 등장확률이 min_support이상인 itemset만 출력
    rules=association_rules(frequent_itemsets, metric="lift", min_threshold=1)
    
    
    def association_rule(rules,i):
        playlist=tag_list[i]
        A=rules[(rules.antecedents=={playlist[0]})]
        for p in range(len(playlist)-1):
            try:
                B=rules[(rules.antecedents=={playlist[p+1]})]
                C=pd.concat([A,B],axis=0).reset_index(drop=True)
                A=C
            except: break
    
        selected=A[['antecedents','consequents','support','confidence','lift']].sort_values(by=['lift'],axis=0,ascending=False) #향상도에 내림차순 정렬
    #selected[selected['lift']>1.5] #향상도 1.5이상만 보여줌

        S=selected[selected['lift']>1.0].reset_index(drop=True)
        taglist=[]
    
        for c in range(len(S.consequents)):
            T=list(S.consequents[c])
            taglist.append(T)
        
        taglist=list(itertools.chain.from_iterable(taglist))   

        Tag=[]
        for v in taglist:
            if v not in Tag:
                Tag.append(v)
     
        return Tag
    
    for i in range(n):
        Tag=association_rule(rules,i)
        print (Tag)
        
    

In [3]:
association_tags(datapath,0.09,5)



[]
[]
[]
[]
[]
